# Voice Analysis

Using Parselmouth, Praat's Voice Report (which is accessible in from its Sound Viewer's menu: Pulses->Voice report) can be reproduced in Python using Parselmouth. This example steps through the process to obtain the identical voice analysis report.

We use an audio recording of someone saying *"Two"*: [2_b.wav](audio/2_b.wav) and pick the voiced audio samples within. Note: Ideally, your acoustic signal should be of a sustained phonation (of your choice of vowel). Anyhow, here is what Praat produces after it runs Voice Analysis:

```
-- Voice report for 1. Sound 2_b --
Date: Mon Feb 22 09:50:04 2021

Time range of SELECTION
   From 0.179649 to 0.387611 seconds (duration: 0.207962 seconds)
Pitch:
   Median pitch: 102.297 Hz
   Mean pitch: 104.189 Hz
   Standard deviation: 15.830 Hz
   Minimum pitch: 84.573 Hz
   Maximum pitch: 153.580 Hz
Pulses:
   Number of pulses: 22
   Number of periods: 21
   Mean period: 9.653643E-3 seconds
   Standard deviation of period: 1.359495E-3 seconds
Voicing:
   Fraction of locally unvoiced frames: 0   (0 / 63)
   Number of voice breaks: 0
   Degree of voice breaks: 0   (0 seconds / 0 seconds)
Jitter:
   Jitter (local): 2.786%
   Jitter (local, absolute): 268.942E-6 seconds
   Jitter (rap): 0.606%
   Jitter (ppq5): 0.657%
   Jitter (ddp): 1.819%
Shimmer:
   Shimmer (local): 11.646%
   Shimmer (local, dB): 1.494 dB
   Shimmer (apq3): 2.097%
   Shimmer (apq5): 5.126%
   Shimmer (apq11): 3.040%
   Shimmer (dda): 6.292%
Harmonicity of the voiced parts only:
   Mean autocorrelation: 0.948082
   Mean noise-to-harmonics ratio: 0.070337
   Mean harmonics-to-noise ratio: 15.956 dB
```

To replicate these results in Python/Parselmouth, we will utilize the following classes therein: `parselmouth.Sound`, `parselmouth.Pitch`, and `parselmouth.PointProcess`.

## Step 0. Get necessary packages

We start out by importing `parselmouth` and other Python packages

In [ ]:
import parselmouth

from os import path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

## Step 1. Read the acoustic data and set analysis range

Fist thing to do is to open and read in the audio file and set analysis window (or domain)

In [ ]:
# Load acoustic data from the WAV file
wavfile = "audio/2_b.wav"
sound = parselmouth.Sound(wavfile)

# Define the window
tlim = (0.179649, 0.387611)

# Plot the waveform and analysis window
plt.plot(sound.xs(), sound.values.T)
plt.axvline(tlim[0],ls=":",c="tab:red")
plt.axvline(tlim[1],ls=":",c="tab:red")
plt.xlim((sound.xmin, sound.xmax))
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.show()

## Step 2: Create `Pitch` and `PointProcess` objects

Next, generate `Pitch` and `PointProcess` objects from `sound`. `Pitch` objects contain measurements of the fundamental frequency and its strength in equispaced frames (default: 33.33-ms wide) while `PointProcess` objects contain a series of time markers, which separates observed voice periods.

Note that there are 2 algorithms to generate `Pitch` object from `Sound` object: `to_pitch_ac()` and `to_pitch_cc()`. The former is based on autocorrelation method while the latter is based on cross-correlation. Praat recommends using the cross-correlation method for voice analysis.

Likewise, there are 3 methods to generate `PointProcess` object from `Pitch` object: (1) purely based on the pitch estimates, (2) use `Sound` samples to obtain more accurate cycle boundaries using cross-correlation, and (3) like (2) but use less-accurate (but faster) peak-picking method. Again, for the best results, use the cross-correlation based method.

In [ ]:
# instantiate Pitch and PointProcess objects
pitch = sound.to_pitch_cc()
pulses = pitch.to_point_process_cc(sound)

# Compute instantaneous fundamental frequency estimates
tb = np.array(pulses)
f0 = 1/np.diff(tb)
tp = (tb[:-1]+tb[1:])/2.0 # center of the cycle

# plot the pitch contour and cycles
plt.subplot(2,1,1,xlim=tlim,ylabel="fundamental frequency (Hz)")
plt.plot(pitch.xs(), pitch.selected_array["frequency"])
plt.plot(tp,f0,'.')
for t in tb:
    plt.axvline(t,ls=":",c="tab:blue")
plt.legend(("`Pitch`","Instantaneous $F_0$"))

plt.subplot(2,1,2,xlim=tlim,ylabel="amplitude",xlabel="time [s]")
plt.plot(sound.xs(), sound.values.T,color="gray")
for t in tb:
    plt.axvline(t,ls=":",c="tab:blue")
plt.legend(("acoustic data", "cycle boundaries",))
plt.show()

## Step 3: Voice Report Header

Now we are ready to produce our own report. First, the trivial stuff at the top...

In [ ]:
print(f"-- Voice report for 1. {path.basename(wavfile)[:-4]} --")
print(f"Date: {datetime.now().strftime('%c')}")
print(f"Time range of SELECTION")
print(f"    From {tlim[0]:6g} to {tlim[1]:6g} seconds (duration: {tlim[1]-tlim[0]:6g} seconds)")

## Step 4: Pitch Entry

Pitch statistics are obtained from `pitch` object. Note `tlim` is unpacked in the function arguments to provide the 2nd (`from_time`) and 3rd `end_time` arguments. 

In [ ]:
print(f"Pitch:")
print(f"   Median pitch: {pitch.get_quantile(0.5,*tlim):0.3f} Hz")
print(f"   Mean pitch: {pitch.get_mean(*tlim):0.3f} Hz")
print(f"   Standard deviation: {pitch.get_standard_deviation(*tlim):0.3f} Hz")
print(f"   Minimum pitch: {pitch.get_minimum(*tlim):0.3f} Hz")
print(f"   Maximum pitch: {pitch.get_maximum(*tlim):0.3f} Hz")

## Step 5: Pulse Entry

Pulse (or cycle) statistics are obtained from `pulses` object. 

Be aware that `PointProcess.get_number_of_points()` always returns the total number of points and not over the analysis window. If this information is critical, extract the target audio data first via `sound.extract_part(...)` then run the rest of the analysis on it.


In [ ]:
print("Pulses:")
print(f"   Number of pulses: {pulses.get_number_of_points()}")
print(f"   Number of periods: {pulses.get_number_of_periods(*tlim)}")
print(f"   Mean period: {pulses.get_mean_period(*tlim):0.6E} seconds")
print(f"   Standard deviation of period: {pulses.get_stdev_period(*tlim):0.6E} seconds")

## Step 6: Voicing Entry

Unvoiced period analysis is performed by `Pitch.get_fraction_of_locally_unvoiced_frames(...)` while
voice break analysis is performed by `PointProcess.get_count_and_fraction_of_voice_breaks(...)`

In [ ]:
lu_frac, lu_num, lu_den = pitch.get_fraction_of_locally_unvoiced_frames(*tlim)
n, deg, Tu, Tt = pulses.get_count_and_fraction_of_voice_breaks(*tlim)

print("Voicing:")
print(f"   Fraction of locally unvoiced frames: {lu_frac}   ({lu_num} / {lu_den})")
print(f"   Number of voice breaks: {n}")
print(f"   Degree of voice breaks: {deg}   ({Tu} seconds / {Tt} seconds)")

## Step 7: Jitter Entry

Jitters are measured by `PointProcess.get_jitter_xxx(...)` methods

In [ ]:
print("Jitter:")
print(f"   Jitter (local): {pulses.get_jitter_local(*tlim)*100:0.3f}%")
print(f"   Jitter (local, absolute): {pulses.get_jitter_local_absolute(*tlim):0.5E} seconds") # shows up as "E-4"  rather than "E-6"
print(f"   Jitter (rap): {pulses.get_jitter_rap(*tlim)*100:0.3f}%")
print(f"   Jitter (ppq5): {pulses.get_jitter_ppq5(*tlim)*100:0.3f}%")
print(f"   Jitter (ddp): {pulses.get_jitter_ddp(*tlim)*100:0.3f}%")

## Step 8: Shimmer Entry

Shimmers are measured by `PointProcess.get_shimmer_xxx(...)` methods and requires the source `Sound` object

In [ ]:
print("Shimmer:")
print(f"   Shimmer (local): {pulses.get_shimmer_local(sound,*tlim)*100:.3f}%")
print(f"   Shimmer (local, dB): {pulses.get_shimmer_local_dB(sound,*tlim):.3f} dB")
print(f"   Shimmer (apq3): {pulses.get_shimmer_local_apq3(sound,*tlim)*100:.3f}%")
print(f"   Shimmer (apq5): {pulses.get_shimmer_local_apq5(sound,*tlim)*100:.3f}%")
print(f"   Shimmer (apq11): {pulses.get_shimmer_local_apq11(sound,*tlim)*100:.3f}%")
print(f"   Shimmer (dda): {pulses.get_shimmer_local_dda(sound,*tlim)*100:.3f}%")

## Step 9: Harmonicity Entry

The last section of the report contains the harmonicity analysis. This analysis is done with `Pitch` object, specifically with its `get_mean_strength(...)` method

In [ ]:
print("Harmonicity of the voiced parts only:")
print(f"   Mean autocorrelation: {pitch.get_mean_strength('ac', *tlim):.6f}")
print(f"   Mean noise-to-harmonics ratio: {pitch.get_mean_strength('nhr', *tlim):.6f}")
print(f"   Mean harmonics-to-noise ratio: {pitch.get_mean_strength('hnr_db', *tlim):.3f}")

## Summary

This notebook presents an example of how to run the full Praat Voice Analysis and print its report as you would see them in Praat UI. 

Obviously, simply replicating what Praat does exactly is not the strength of `Parselmouth`. With Python and `Parselmouth` package, you can:

- Compute and record only the voice measures of your interest
- Directly write the measurements to a file
- Batch process multiple input files
- Combine Praat parameters with other voice parameters
- Add preprocessing algorithm to auto-select analysis window (i.e., auto-picking `tlim`)

Happy Parselmouth'ing!